In [36]:
import numpy as np
import pandas as pd
import os, warnings, math

from functions_tony import reduce_mem_usage

In [2]:
train = pd.read_csv('/home/antonio.correia/Documents/Kaggle/ASHRAE_energy/train.csv')
test = pd.read_csv('/home/antonio.correia/Documents/Kaggle/ASHRAE_energy/test.csv')
meta = pd.read_csv('/home/antonio.correia/Documents/Kaggle/ASHRAE_energy/building_metadata.csv')
weather_train = pd.read_csv('/home/antonio.correia/Documents/Kaggle/ASHRAE_energy/weather_train.csv')
weather_test = pd.read_csv('/home/antonio.correia/Documents/Kaggle/ASHRAE_energy/weather_test.csv')

In [49]:
train_df = train.set_index('building_id')\
.join(meta.set_index('building_id'), on='building_id', how='left').reset_index()

In [50]:
train_df = train_df.set_index(['site_id','timestamp'])\
.join(weather_train.set_index(['site_id','timestamp']),\
      on=['site_id','timestamp'], how='left').reset_index()

In [51]:
test_df = test.set_index('building_id')\
.join(meta.set_index('building_id'), on='building_id', how='left').reset_index()

test_df = test_df.set_index(['site_id','timestamp'])\
.join(weather_test.set_index(['site_id','timestamp']),\
      on=['site_id','timestamp'], how='left').reset_index()

In [38]:
train_df.dtypes

site_id                 int64
timestamp              object
building_id             int64
meter                   int64
meter_reading         float64
primary_use            object
square_feet             int64
year_built            float64
floor_count           float64
air_temperature       float64
cloud_coverage        float64
dew_temperature       float64
precip_depth_1_hr     float64
sea_level_pressure    float64
wind_direction        float64
wind_speed            float64
dtype: object

In [39]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
site_id,20216100.0,7.992232,5.099060,0.0,3.0,9.000,13.000,15.0
building_id,20216100.0,799.278000,426.913309,0.0,393.0,895.000,1179.000,1448.0
meter,20216100.0,0.662441,0.930992,0.0,0.0,0.000,1.000,3.0
meter_reading,20216100.0,2117.121076,153235.624998,0.0,18.3,78.775,267.984,21904700.0
square_feet,20216100.0,107782.981330,117142.443648,283.0,32527.0,72709.000,139113.000,875000.0
year_built,8088455.0,1968.276736,30.208153,1900.0,1951.0,1969.000,1993.000,2017.0
floor_count,3506933.0,4.184848,4.008277,1.0,1.0,3.000,6.000,26.0
air_temperature,20119442.0,15.987953,10.947293,-28.9,8.6,16.700,24.100,47.2
cloud_coverage,11390735.0,1.900423,2.402909,0.0,0.0,0.000,4.000,9.0
dew_temperature,20115960.0,7.747429,10.178667,-35.0,0.0,8.900,16.100,26.1


In [40]:
reduce_mem_usage(train_df).describe().T

Mem. usage decreased to 886.86 Mb (64.1% reduction)


,count,mean,std,min,25%,50%,75%,max
site_id,20216100.0,7.992232,5.099060,0.00000,3.000000,9.000000,13.000000,1.500000e+01
building_id,20216100.0,799.278000,426.913309,0.00000,393.000000,895.000000,1179.000000,1.448000e+03
meter,20216100.0,0.662441,0.930992,0.00000,0.000000,0.000000,1.000000,3.000000e+00
meter_reading,20216100.0,2117.121582,153235.625000,0.00000,18.299999,78.775002,267.984009,2.190470e+07
square_feet,20216100.0,107782.981330,117142.443648,283.00000,32527.000000,72709.000000,139113.000000,8.750000e+05
year_built,8088455.0,NaN,NaN,1900.00000,1951.000000,1969.000000,1993.000000,2.017000e+03
floor_count,3506933.0,NaN,0.000000,1.00000,1.000000,3.000000,6.000000,2.600000e+01
air_temperature,20119442.0,NaN,0.000000,-28.90625,8.601562,16.703125,24.093750,4.718750e+01
cloud_coverage,11390735.0,NaN,0.000000,0.00000,0.000000,0.000000,4.000000,9.000000e+00
dew_temperature,20115960.0,NaN,0.000000,-35.00000,0.000000,8.898438,16.093750,2.609375e+01


Downcasting fields affects some of the dataframe's descriptive statistics. We will only downcast the following fields:
`[site_id','building_id','meter','meter_reading','square_feet']`

In [41]:
train_df = train.set_index('building_id')\
.join(meta.set_index('building_id'), on='building_id', how='left').reset_index()

train_df = train_df.set_index(['site_id','timestamp'])\
.join(weather_train.set_index(['site_id','timestamp']),\
      on=['site_id','timestamp'], how='left').reset_index()

In [72]:
downcast_fields = ['site_id','building_id','meter','meter_reading','square_feet']
train_df_dc_fields = reduce_mem_usage(train_df[downcast_fields])

downcast_fields.remove('meter_reading')
test_df_dc_fields = reduce_mem_usage(test_df[downcast_fields])

Mem. usage decreased to 231.35 Mb (70.0% reduction)
Mem. usage decreased to 318.13 Mb (75.0% reduction)


In [88]:
for c in train_df_dc_fields.columns:
    train_df[c] = train_df_dc_fields[c]
    
for c in test_df_dc_fields.columns:
    test_df[c] = test_df_dc_fields[c]

In [90]:
train_df.to_pickle("/home/antonio.correia/Documents/Kaggle/ASHRAE_energy/train_df.pkl")
test_df.to_pickle("/home/antonio.correia/Documents/Kaggle/ASHRAE_energy/test_df.pkl")

In [32]:
weather_train[(weather_train['site_id']==15) & (weather_train['timestamp']>='2016-05-05 00:00:00')][0:24]

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
134187,15,2016-05-05 00:00:00,10.6,4.0,8.9,NaN,1006.2,110.0,1.5
134188,15,2016-05-05 01:00:00,9.4,NaN,7.8,NaN,1006.8,110.0,2.6
134189,15,2016-05-05 02:00:00,8.3,4.0,7.2,NaN,1007.6,110.0,2.1
134190,15,2016-05-05 03:00:00,9.4,NaN,7.2,NaN,1007.3,120.0,2.1
134191,15,2016-05-05 04:00:00,9.4,NaN,7.8,NaN,1007.2,90.0,1.5
134192,15,2016-05-05 06:00:00,8.9,NaN,7.2,NaN,1007.1,70.0,1.5
134193,15,2016-05-05 07:00:00,8.3,NaN,6.1,NaN,1007.2,80.0,2.1
134194,15,2016-05-05 08:00:00,7.8,NaN,5.6,NaN,1007.5,100.0,2.1
134195,15,2016-05-05 09:00:00,7.2,NaN,5.0,NaN,1007.3,70.0,2.6
134196,15,2016-05-05 10:00:00,6.1,4.0,4.4,NaN,1007.6,70.0,2.6


Sites which are colder during the day than at night:
 - [0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15]
 - However, some of these sites are not entirely colder at day than at night. It can be that the real timezone is not the one disclosed on the timestamp field